In [41]:
import numpy as np
# Model design
import agentpy as ap
# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython


    
class Roads(ap.Agent):
    def setup(self):
        self.road = 1
        self.condition = 1
    
class Vehicle(ap.Agent):
    def setup(self):
        self.grid = self.model.grid
        self.crossed = False
        self.road = 1
        self.pos = [0, 0]
        self.side = [1, 0]
        self.speed = 1
       
    def route(self):
        self.pos = self.grid.positions[self]
        if self.pos[1] == 0:
            return 'HORIZONTAL'
        return 'VERTICAL'

    def direction(self):
        self.pos = self.grid.positions[self]
        if self.pos[1] == 0:
            self.side = [0, 1]

    def movement(self):
        self.direction()
        return (self.side[0] * self.speed, self.side[1] * self.speed)
    
class StopSign(ap.Agent):
    def setup(self):
        self.grid = self.model.grid
        self.status = 1
        self.road = 3
        self.pos = [0, 0]
        self.route = ''

    def positions(self):
        self.pos = self.grid.positions[self]

    def change_state(self):
        self.positions()
        tGrid = self.p['Grid']
        self.status = 2
        self.road = 2
        if self.model.n_cars_1 <= self.model.n_cars_2:
            if self.pos[1] == int((tGrid / 2) + 1):
                self.status = 0
                self.road = 4
                
        else:
            if self.pos[0] == int((tGrid / 2) + 1):
                self.status = 0
                self.road = 4
 

class IntersectionModel(ap.Model):
    def setup(self):
        self.con = 0
        tGrid = self.p['Grid']
        self.grid = ap.Grid(self, [tGrid] * 2, torus = True, track_empty = True)
        self.n_cars_1 = 0
        self.n_cars_2 = 0
        n_vehicles = self.p['Vehicles']
        n_roads = tGrid * 2
        self.vehicles = ap.AgentList(self, n_vehicles, Vehicle)
        self.road = ap.AgentList(self, n_roads, Roads)
        self.stop_sign = ap.AgentList(self, 2, StopSign)
        self.vehicles.grid = self.grid
        road_pos = []
        tGrid = self.p['Grid']
        
        for i in range(tGrid * 2):
            if i <= tGrid-1:
                road_pos.append((i, int(tGrid / 2)))
            else:
                road_pos.append((int(tGrid / 2), i - tGrid))
            
        self.grid.add_agents(self.stop_sign, positions=[(int((tGrid / 2) - 1), int((tGrid / 2) + 1)), (int((tGrid / 2) + 1), int((tGrid / 2) - 1))])
        for semaforo in self.stop_sign:
            if self.grid.positions[semaforo][1] == 0:
                semaforo.route = 'VERTICAL'
            else: semaforo.route = 'HORIZONTAL'
                
        vehicles_positions=[]
        for i in range (1,n_vehicles+1):
            if i % 2 == 0:
                vehicles_positions.append((int(tGrid / 2), 0))
            else:
                vehicles_positions.append((0, int(tGrid / 2)))


    def step(self):
        tGrid = self.p['Grid']
        n_vehicles = self.p['Vehicles']
        if self.con == 0:
            vehicles_positions=[]
            positions=[(0, int(tGrid / 2)), (int(tGrid / 2), 0)]
            for i in range(n_vehicles):
                vehicles_positions.append(positions[np.random.randint(0, 2)])
            new_car = ap.AgentList(self,1,Vehicle)
            self.grid.add_agents(self.vehicles,vehicles_positions)
            self.con += 1

        n_movements = 0
        move_cars = 0
        movimiento = True
        state = False
        for car in self.vehicles :
            agent_pos = self.grid.positions[car]
            for i in range(1,int((tGrid/2))):
                if (int(tGrid / 2) == agent_pos[0])and(int(i) == agent_pos[1]):
                    self.n_cars_1 += 1
                    state = True
                if  (int(tGrid/2) == agent_pos[1])and(int(i) == agent_pos[0]):
                    self.n_cars_2 += 1
                    state = True
        if state:
            for semaforo in self.stop_sign:   
                if state:
                    semaforo.change_state()
                else:
                    semaforo.status = 1
                    semaforo.road = 3

        for agent in self.grid.agents:
            agent_pos = self.grid.positions[agent]
            movimiento = True
            if agent.type == 'Vehicle':
                for neighbor in self.grid.neighbors(agent):
                    if agent.route() == 'VERTICAL':
                        if self.grid.positions[neighbor][1] == agent_pos[1] + 1 and self.grid.positions[neighbor][0] == agent_pos[0]:
                            if  neighbor.type == 'Vehicle':    
                                movimiento = False
                                break
                                
                        if self.grid.positions[neighbor][1] ==agent_pos[1] and self.grid.positions[neighbor][0]==agent_pos[0]+1:
                            if  neighbor.type == 'StopSign':
                                if neighbor.status == 2:
                                    movimiento = False
                                    break
                                else:
                                    self.n_cars_1 -= 1
                                    break
                        
                    if agent.route() == 'VERTICAL':
                        if self.grid.positions[neighbor][0] == agent_pos[0] + 1 and self.grid.positions[neighbor][1] == agent_pos[1]:
                            if  neighbor.type == 'Vehicle':
                                movimiento = False
                                break
                                
                        if self.grid.positions[neighbor][0] == agent_pos[0] and self.grid.positions[neighbor][1] == agent_pos[1] + 1:
                            if  neighbor.type == 'StopSign':
                                if neighbor.status == 2:
                                    movimiento = False
                                    break
                                else:
                                    self.n_cars_2 -= 1
                                    break
                if movimiento:
                    self.grid.move_by(agent,agent.movement())

parameters = {
    'Vehicles': 20,
    'steps': 300,
    'Grid' : 30
}

def animation_plot(model, ax):
    attr_grid = model.grid.attr_grid('road')
    #Colores = 0 : verde fuerte, 1 : gris, 2 : rojo, 3 : amarillo, 4 : verde claro
    color_dict = {0: '#1C542D', 1: '#e5e5e5', 2: '#d62c2c', 3: '#FFFF00', 4: '#21d41e', None: '#7FC97F'} 
    ap.gridplot(attr_grid, ax = ax, color_dict = color_dict, convert = True)


fig, ax = plt.subplots()
model = IntersectionModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=30))